In [1]:
from jupyter_dash import JupyterDash     # pip install dash  
from dash import Dash, html, dcc
from dash.dependencies import Output, Input
from datetime import datetime            # converting UNIX timestamps
from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar
from wordcloud import WordCloud          # pip install wordcloud
import webbrowser
import pandas as pd
import calendar

#This is for lotties
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

# JSON URL lotties
LOTTIE_URLS = {
    'connections': "https://lottie.host/8c872553-bf3d-4ef4-9739-01b2718d6c09/q6LtyMNZ28.json",
    'msg_in': "https://assets9.lottiefiles.com/packages/lf20_8wREpI.json",
    'msg_out': "https://assets2.lottiefiles.com/packages/lf20_Cc8Bpg.json",
    'reactions': "https://assets2.lottiefiles.com/packages/lf20_nKwET0.json",
    # Instagram: Stories/Following, LinkedIn: Companies
    'companies_stories': "https://lottie.host/7b12b34c-bb3e-4a48-9ca7-71274eaed86d/gnor7nkdhw.json", 
}

#Loading CSV LINKEDIN

# --- LinkedIn Data Load ---
df_li_cnt = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Connections.csv", on_bad_lines='skip')
df_li_invite = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Invitations.csv", on_bad_lines='skip')
df_li_react = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Reactions.csv", on_bad_lines='skip')
df_li_msg = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/messages.csv", on_bad_lines='skip')

# Process LinkedIn Dates
if "Connected On" in df_li_cnt.columns:
    df_li_cnt["Connected On"] = pd.to_datetime(df_li_cnt["Connected On"], errors='coerce')
    df_li_cnt["month"] = df_li_cnt["Connected On"].dt.month.apply(lambda x: calendar.month_abbr[int(x)] if pd.notnull(x) and int(x) in range(1,13) else "")
else:
    df_li_cnt["Connected On"] = pd.NaT

if "Sent At" in df_li_invite.columns:
    df_li_invite["Sent At"] = pd.to_datetime(df_li_invite["Sent At"], errors='coerce')
else:
    df_li_invite["Sent At"] = pd.NaT

if "Date" in df_li_react.columns:
    df_li_react["Date"] = pd.to_datetime(df_li_react["Date"], errors='coerce')
else:
    df_li_react["Date"] = pd.NaT

if "DATE" in df_li_msg.columns:
    df_li_msg["DATE"] = pd.to_datetime(df_li_msg["DATE"], errors='coerce')
else:
    df_li_msg["DATE"] = pd.NaT


# --- Instagram Data Load ---
df_ig_cnt = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramConnectionsFollowing.csv", on_bad_lines='skip')
df_ig_invite = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/Invitations.csv", on_bad_lines='skip')
df_ig_react = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramReactions.csv", on_bad_lines='skip')
df_ig_msg = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramMessages.csv", on_bad_lines='skip')

# Process Instagram Dates
if "Connected On" in df_ig_cnt.columns:
    df_ig_cnt["Connected On"] = pd.to_datetime(df_ig_cnt["Connected On"], errors='coerce')
    df_ig_cnt["month"] = df_ig_cnt["Connected On"].dt.month.apply(lambda x: calendar.month_abbr[int(x)] if pd.notnull(x) and int(x) in range(1,13) else "")
else:
    df_ig_cnt["Connected On"] = pd.NaT

if "Sent At" in df_ig_invite.columns:
    df_ig_invite["Sent At"] = pd.to_datetime(df_ig_invite["Sent At"], errors='coerce')
else:
    df_ig_invite["Sent At"] = pd.NaT

if "DateTimeStamp" in df_ig_react.columns:
    df_ig_react["DateTimeStamp"] = pd.to_datetime(df_ig_react["DateTimeStamp"])
else:
    df_ig_react["DateTimeStamp"] = pd.NaT

# Note: Instagram messages column name seems inconsistent, using what was implied as the timestamp column
if "messages__timestamp_ms" in df_ig_msg.columns:
    # Assuming the timestamp is milliseconds since epoch and needs conversion
    # Note: Your original code had df_msg["DATE"] = pd.to_datetime(df_msg["DATE"], errors='coerce') which is confusing
    # We will try to convert the timestamp_ms column
    try:
        df_ig_msg["messages__timestamp_ms"] = pd.to_datetime(df_ig_msg["messages__timestamp_ms"], unit='ms', errors='coerce')
    except:
        df_ig_msg["messages__timestamp_ms"] = pd.NaT
else:
    df_ig_msg["messages__timestamp_ms"] = pd.NaT


# --- Global Date Range Calculation (Combined) ---
min_date_li = df_li_cnt["Connected On"].min().date() if df_li_cnt["Connected On"].notna().any() else date(2018, 1, 1)
max_date_li = df_li_cnt["Connected On"].max().date() if df_li_cnt["Connected On"].notna().any() else date.today()

min_date_ig = df_ig_cnt["Connected On"].min().date() if df_ig_cnt["Connected On"].notna().any() else date(2018, 1, 1)
max_date_ig = df_ig_cnt["Connected On"].max().date() if df_ig_cnt["Connected On"].notna().any() else date.today()

# Use the earlier start date and later end date as global bounds for the date picker
MIN_DATE = min(min_date_li, min_date_ig)
MAX_DATE = max(max_date_li, max_date_ig)

# --- 3. APP INITIALIZATION & LAYOUT FUNCTIONS ---

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX], suppress_callback_exceptions=True)

def create_card(lottie_url, title, id_prefix):
    """Helper function to create a standard dashboard card."""
    return dbc.Col([
        dbc.Card([
            dbc.CardHeader(
                html.Div(
                    Lottie(options=options, url=lottie_url),
                    style={"width": "180px", "height": "150px", "margin": "0 auto", "display": "block"}
                )
            ),
            dbc.CardBody([
                html.H6(title),
                html.H2(id=f'{id_prefix}-content', children="000")
            ], style={'textAlign':'center'})
        ], className="h-100")
    ], width=2)


def serve_linkedin_layout():
    """Generates the layout for the LinkedIn Dashboard page."""
    prefix = 'li'
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardImg(src='assets/linkedin-logo2.png', className='p-3')
                ],className='mb-2'),
                dbc.Card([
                    dbc.CardBody([
                        dbc.CardLink("Project Source", target="_blank", href="https://github.com/busyizzybee/socialMediaEngagement")
                    ])
                ]),
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4("LinkedIn Engagement Analysis", className="card-title"),
                        html.P("Filter data by connection date range:", className="card-text"),
                        dcc.DatePickerSingle(id=f'{prefix}-date-picker-start', date=MIN_DATE, className='ml-5'),
                        dcc.DatePickerSingle(id=f'{prefix}-date-picker-end', date=MAX_DATE, className='mb-2 ml-2'),
                    ])
                ], color="info"),
            ], width=10),
        ],className='mb-2 mt-2'),
        dbc.Row([
            create_card(LOTTIE_URLS['connections'], 'Connections', f'{prefix}-connections'),
            create_card(LOTTIE_URLS['companies_stories'], 'Unique Companies', f'{prefix}-companies'),
            create_card(LOTTIE_URLS['msg_in'], 'Invites Received', f'{prefix}-msg-in'),
            create_card(LOTTIE_URLS['msg_out'], 'Invites Sent', f'{prefix}-msg-out'),
            create_card(LOTTIE_URLS['reactions'], 'Reactions', f'{prefix}-reactions'),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-line-chart', figure={}, config={'displayModeBar':False})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-bar-chart', figure={}, config={'displayModeBar':False})])), width=6),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-pie-chart', figure={})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([
                dcc.Graph(id=f'{prefix}-wordcloud', figure={}, config={'displayModeBar': False}),
                html.P("The word cloud shows the most common job titles among connections in the selected period.", 
                       style={'textAlign': 'center', 'fontStyle': 'italic', 'fontSize': '14px'})
            ])), width=6),
        ], className='mb-2'),
    ], fluid=True)


def serve_instagram_layout():
    """Generates the layout for the Instagram Dashboard page."""
    prefix = 'ig'
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardImg(src='assets/instagram-logo.png', className='p-3')
                ],className='mb-2'),
                dbc.Card([
                    dbc.CardBody([
                        dbc.CardLink("LinkedIn Dash", target="_blank", href="/linkedin")
                    ])
                ]),
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4("Instagram Engagement Analysis", className="card-title"),
                        html.P("Filter data by connection date range:", className="card-text"),
                        dcc.DatePickerSingle(id=f'{prefix}-date-picker-start', date=MIN_DATE, className='ml-5'),
                        dcc.DatePickerSingle(id=f'{prefix}-date-picker-end', date=MAX_DATE, className='mb-2 ml-2'),
                    ])
                ], color="thistle"),
            ], width=10),
        ],className='mb-2 mt-2'),
        dbc.Row([
            create_card(LOTTIE_URLS['connections'], 'Followers', f'{prefix}-followers'),
            create_card(LOTTIE_URLS['companies_stories'], 'Following', f'{prefix}-following'),
            create_card(LOTTIE_URLS['msg_in'], 'DMs Received', f'{prefix}-msg-in'),
            create_card(LOTTIE_URLS['msg_out'], 'DMs Sent', f'{prefix}-msg-out'),
            create_card(LOTTIE_URLS['reactions'], 'Reactions/Likes', f'{prefix}-reactions'),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-line-chart', figure={}, config={'displayModeBar':False})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-bar-chart', figure={}, config={'displayModeBar':False})])), width=6),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-pie-chart', figure={})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-wordcloud', figure={})])), width=6),
        ], className='mb-2'),
    ], fluid=True)


# Main App Layout with Routing
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.NavbarSimple(
        children=[
            dbc.NavItem(
                dbc.NavLink(
                    "About Us", 
                    href="/aboutus", 
                    # Embedded style for larger link text
                    style={'fontSize': '1.15rem', 'marginRight': '1rem'} 
                )
            ),
            dbc.NavItem(
                dbc.NavLink(
                    "LinkedIn Dashboard", 
                    href="/linkedin", 
                    style={'fontSize': '1.15rem', 'marginRight': '1rem'}
                )
            ),
            dbc.NavItem(
                dbc.NavLink(
                    "Instagram Dashboard", 
                    href="/instagram", 
                    style={'fontSize': '1.15rem', 'marginRight': '1rem'}
                )
            ),
        ],
        brand=html.Span(
            "Social Media Analytics", 
            # Embedded style for larger, bolder brand text
            style={'fontSize': '1.75rem', 'fontWeight': '700', 'paddingLeft': '1rem'}
        ), 
        brand_href="/",
        color="primary",
        dark=True,
        className="mb-4",
    ),
    html.Div(id='page-content')
])

# --- 4. ROUTING CALLBACK ---
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/linkedin' or pathname == '/':
        return serve_linkedin_layout()
    elif pathname == '/instagram':
        return serve_instagram_layout()
    elif pathname == '/aboutus':
        return serve_aboutus_layout()
    else:
        return html.Div(dbc.Alert(f"404 - Page not found: {pathname}", color="danger"))


# --- 5. CALLBACKS (ABOUT US) ---
def serve_aboutus_layout():
    """Layout for the About Us page, with left-aligned logo and a visibly centered title."""
    
    GITHUB_URL = "https://github.com/busyizzybee/socialMediaEngagement"
    
    # Define a common style for the profile pictures
    profile_img_style = {
        "width": "120px",    
        "height": "120px",   
        "borderRadius": "50%", 
        "objectFit": "cover",  
        "marginBottom": "10px",
        "border": "2px solid #007bff" 
    }

    # Style for justified text
    justify_style = {'textAlign': 'justify'}

    return dbc.Container([
        # Row 1: Logo (Left-aligned, Closer to Center) and Main Title (Visibly Centered)
        dbc.Row([
            # New Spacer Column (pushes logo inward)
            dbc.Col(width=1), 
            
            # Column for Logo (width="auto" keeps it small)
            dbc.Col([
                html.Img(
                    src='/assets/Connectly.png', 
                    height="180px", 
                    style={"display": "block"} 
                ),
            ], width="auto", className="d-flex align-items-center"), 

            # Column for Title: width=7 and ml-auto to shift the title to the right, 
            # balancing the space taken by the logo and centering it visually.
            dbc.Col([
                html.H2(
                    "SOCIAL MEDIA ANALYSIS MANAGEMENT PLATFORM", 
                    className="text-center text-primary" 
                ), 
            ], width=7, className="d-flex align-items-center ml-auto"), 
            
            # Final empty column to push the title over
            dbc.Col(width=2) 

        ], className="mb-5 mt-5 d-flex align-items-center border-bottom pb-4"), 

        # Description Part (Justified - width=9) - UNCHANGED
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    dbc.CardBody([
                        html.H4("PROJECT OVERVIEW", className="card-title"),
                        html.P(
                            "This Social Media Analytics Dashboard was developed to visualize and analyze "
                            "personal social media engagement data from **LinkedIn** and **Instagram**. "
                            "The goal is to provide a comprehensive, data-driven view of connection trends, "
                            "messaging activity, and content reactions. By leveraging data from these "
                            "platforms, users can gain insights to improve their digital presence and "
                            "professional networking strategies.",
                            className="card-text",
                            style=justify_style 
                        ),
                        html.P(
                            ["Check out the full source code and documentation on GitHub: ",
                             html.A("Project Repository", href=GITHUB_URL, target="_blank", className="text-info")
                            ], 
                            className="card-text mt-3"
                        )
                    ])
                )
            ], width=9, className="mx-auto mb-4"), 
        ]),
        
        # Team & Development Header (Centered) - UNCHANGED
        dbc.Row([
            dbc.Col([
                html.H3("TEAM & DEVELOPMENT", className="text-center mb-3 mt-3"),
            ], width=12, className="mx-auto mt-5"),
        ]),
        
        # Row 3: Team Members (3 members across the top) - UNCHANGED
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Blessy.png', style=profile_img_style), className="d-flex justify-content-center"), 
                html.H5("BLESSY SOLAS", className="card-title text-success"),
                html.P("Lead Developer", className="card-text"),
                html.P("Primary Data Scientist & Dash Implementer", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
            
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Neo.png', style=profile_img_style), className="d-flex justify-content-center"), 
                html.H5("NEO RAY SERRANO", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Data Acquisition & Processing", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),

            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Gerald.png', style=profile_img_style), className="d-flex justify-content-center"), 
                html.H5("GERALD ELLI RAMOS", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Data Analysis & Reporting", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
        ], className="d-flex justify-content-center", style={'width': '100%'}), 

        # Row 4: Team Members (2 members centered below) - UNCHANGED
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Justin.png', style=profile_img_style), className="d-flex justify-content-center"), 
                html.H5("JUSTIN AARON SALONGA", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Visualization & Dashboard Design", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),

            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Sean.png', style=profile_img_style), className="d-flex justify-content-center"), 
                html.H5("SEAN SOLIVEN", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Dash Implementation & Deployment", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
            
        ], className="d-flex justify-content-center mx-auto", style={'width': '85%'}), 
        
        # Row 5: Frameworks (Centered) - UNCHANGED
        dbc.Row([
            dbc.Col([
                html.H3("FRAMEWORKS", className="text-center mb-3 mt-4"),
                dbc.Card(
                    dbc.CardBody([
                        html.H5("Core Technologies", className="card-title text-success"),
                        html.P("Python, Dash, Plotly, Pandas, Dash Bootstrap Components", className="card-text"),
                        html.P("Built for speed and clean visualization.", className="card-text-small text-muted"),
                    ]),
                    className="text-center"
                )
            ], width=6, className="mx-auto mb-4"), 
        ]),

        # Final Note - UNCHANGED
        dbc.Row([
            dbc.Col(
                html.P(
                    "Note: This dashboard is a personal analytics tool. Data displayed is based on the user's downloadable archive data.",
                    style={'textAlign': 'center', 'fontStyle': 'italic', 'fontSize': '12px'},
                    className="mt-4"
                ), width=12)
        ])
    ], fluid=True)

# --- 6. CALLBACKS (LINKEDIN DASHBOARD) ---

# Callback for LinkedIn Small Cards
@app.callback(
    Output('li-connections-content','children'),
    Output('li-companies-content','children'),
    Output('li-msg-in-content','children'),
    Output('li-msg-out-content','children'),
    Output('li-reactions-content','children'),
    [Input('li-date-picker-start','date'),
     Input('li-date-picker-end','date')],
)
def update_li_small_cards(start_date, end_date):
    if not (start_date and end_date):
        return 0, 0, 0, 0, 0
    
    # Connections
    dff_c = df_li_cnt.copy()
    dff_c = dff_c[(dff_c['Connected On'] >= start_date) & (dff_c['Connected On'] <= end_date)]
    conctns_num = len(dff_c)
    compns_num = len(dff_c['Company'].dropna().unique())

    # Invitations
    dff_i = df_li_invite.copy()
    dff_i = dff_i[(dff_i['Sent At'] >= start_date) & (dff_i['Sent At'] <= end_date)]
    in_num = len(dff_i[dff_i['Direction'] == 'INCOMING'])
    out_num = len(dff_i[dff_i['Direction'] == 'OUTGOING'])

    # Reactions
    dff_r = df_li_react.copy()
    dff_r = dff_r[(dff_r['Date'] >= start_date) & (dff_r['Date'] <= end_date)]
    reactns_num = len(dff_r)

    return conctns_num, compns_num, in_num, out_num, reactns_num

# Callback for LinkedIn Line Chart
@app.callback(
    Output('li-line-chart','figure'),
    [Input('li-date-picker-start','date'),
     Input('li-date-picker-end','date')],
)
def update_li_line(start_date, end_date):
    if not (start_date and end_date):
        return {}

    dff = df_li_cnt.copy()
    dff = dff[(dff['Connected On'].astype(str) >= start_date) & (dff['Connected On'].astype(str) <= end_date)]
    month_counts = dff['month'].value_counts()
    
    # Re-order months for correct plotting
    month_order = list(calendar.month_abbr)[1:]
    plot_df = pd.DataFrame({'month': month_order}).merge(
        month_counts.rename('Total connections').reset_index(),
        on='month',
        how='left'
    ).fillna(0)
    plot_df['month'] = pd.Categorical(plot_df['month'], categories=month_order, ordered=True)
    plot_df = plot_df.sort_values('month')

    fig_line = px.line(
        plot_df, x='month', y='Total connections', template='ggplot2', title="Total Connections by Month",
        labels={'month': 'Month', 'Total connections': 'Connections'}
    )
    fig_line.update_traces(mode="lines+markers", fill='tozeroy', line={'color':'blue'})
    fig_line.update_layout(margin=dict(l=20, r=20, t=30, b=20), xaxis_title=None)
    return fig_line

# Callback for LinkedIn Bar Chart (Top Companies)
@app.callback(
    Output('li-bar-chart','figure'),
    [Input('li-date-picker-start','date'),
     Input('li-date-picker-end','date')],
)
def update_li_bar(start_date, end_date):
    if not (start_date and end_date):
        return {}
        
    dff = df_li_cnt.copy()
    dff = dff[(dff['Connected On'].astype(str) >= start_date) & (dff['Connected On'].astype(str) <= end_date)]
    dff = dff.dropna(subset=['Company'])
    company_counts = dff['Company'].value_counts().head(6).reset_index()
    company_counts.columns = ['Company', 'Total connections']

    fig_bar = px.bar(
        company_counts, x='Total connections', y='Company', template='ggplot2', orientation='h', 
        title="Top 6 Companies by Connections",
        labels={'Total connections': 'Connections', 'Company': 'Company'}
    )
    fig_bar.update_yaxes(tickangle=0)
    fig_bar.update_layout(margin=dict(l=20, r=20, t=30, b=20), yaxis_title=None)
    fig_bar.update_traces(marker_color='darkblue')

    return fig_bar

# Callback for LinkedIn Pie Chart (Messages Sent & Received)
@app.callback(
    Output('li-pie-chart','figure'),
    [Input('li-date-picker-start','date'),
     Input('li-date-picker-end','date')],
)
def update_li_pie(start_date, end_date):
    if not (start_date and end_date):
        return {}

    dff = df_li_msg.copy()
    dff = dff[(dff['DATE'].astype(str) >= start_date) & (dff['DATE'].astype(str) <= end_date)]

    # Assuming 'Adam Schroeder' is the user, as specified in your original code
    msg_sent = len(dff[dff['FROM'] == 'Adam Schroeder'])
    msg_rcvd = len(dff[dff['FROM'] != 'Adam Schroeder'])
    
    if msg_sent + msg_rcvd == 0:
        return {}

    fig_pie = px.pie(names=['Sent','Received'], values=[msg_sent, msg_rcvd],
                     template='ggplot2', title="Messages Sent & Received"
    )
    fig_pie.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_pie.update_traces(marker_colors=['#0077b5','#00a0dc']) # LinkedIn colors

    return fig_pie

# Callback for LinkedIn Word Cloud (Positions)
@app.callback(
    Output('li-wordcloud','figure'),
    [Input('li-date-picker-start','date'),
     Input('li-date-picker-end','date')],
)
def update_li_wordcloud(start_date, end_date):
    if not (start_date and end_date):
        return {}

    dff = df_li_cnt.copy()
    dff = dff[(dff['Connected On'].astype(str) >= start_date) & (dff['Connected On'].astype(str) <= end_date)]
    dff_positions = dff['Position'].dropna().astype(str)

    if dff_positions.empty:
         # Return a blank image placeholder if no data is found
        return px.imshow(np.zeros((10,10)), color_continuous_scale='gray', title="No Position Data Found").update_xaxes(visible=False).update_yaxes(visible=False).update_layout(margin=dict(l=20, r=20, t=40, b=20))

    my_wordcloud = WordCloud(background_color='white', width=800, height=400, colormap='winter').generate(' '.join(dff_positions))
    img_array = my_wordcloud.to_array()

    fig_wordcloud = px.imshow(img_array, title="Most Frequent Job Titles")
    fig_wordcloud.update_layout(margin=dict(l=20, r=20, t=40, b=20), xaxis_visible=False, yaxis_visible=False)

    return fig_wordcloud


# --- 7. CALLBACKS (INSTAGRAM DASHBOARD) ---

# Callback for Instagram Small Cards
@app.callback(
    Output('ig-followers-content','children'),
    Output('ig-following-content','children'),
    Output('ig-msg-in-content','children'),
    Output('ig-msg-out-content','children'),
    Output('ig-reactions-content','children'),
    [Input('ig-date-picker-start','date'),
     Input('ig-date-picker-end','date')],
)
def update_ig_small_cards(start_date, end_date):
    if not (start_date and end_date):
        return 0, 0, 0, 0, 0
    
    # Followers/Following (Connections)
    dff_c = df_ig_cnt.copy()
    dff_c = dff_c[(dff_c['Connected On'].astype(str) >= start_date) & (dff_c['Connected On'].astype(str) <= end_date)]

    # Assuming df_ig_cnt contains only 'Following' data based on the column name "InstagramConnectionsFollowing.csv"
    # Total Following count (Used for 'Following' card)
    following_num = len(dff_c)
    
    # You are missing the 'Followers' dataset, so we will use a unique count of followed users as a proxy for engagement.
    # We will use 'following_num' for the 'Following' card and estimate 'Followers' based on invites or use a placeholder.
    # Since the Instagram data provided doesn't have a clean follower/following distinction, we'll use:
    followers_num = len(df_ig_cnt['value'].unique()) # Unique users followed

    # Invitations (Unclear mapping in IG data, using LinkedIn definitions: INCOMING/OUTGOING)
    dff_i = df_ig_invite.copy()
    dff_i = dff_i[(dff_i['Sent At'].astype(str) >= start_date) & (dff_i['Sent At'].astype(str) <= end_date)]
    in_num = len(dff_i[dff_i['Direction'] == 'INCOMING'])
    out_num = len(dff_i[dff_i['Direction'] == 'OUTGOING'])
    
    # Reactions
    dff_r = df_ig_react.copy()
    dff_r = dff_r[(dff_r['DateTimeStamp'].astype(str) >= start_date) & (dff_r['DateTimeStamp'].astype(str) <= end_date)]
    reactns_num = len(dff_r)

    # Note: 'Followers' is used for the first card, 'Following' for the second card.
    return followers_num, following_num, in_num, out_num, reactns_num

# Callback for Instagram Line Chart
@app.callback(
    Output('ig-line-chart','figure'),
    [Input('ig-date-picker-start','date'),
     Input('ig-date-picker-end','date')],
)
def update_ig_line(start_date, end_date):
    if not (start_date and end_date):
        return {}

    dff = df_ig_cnt.copy()
    dff = dff[(dff['Connected On'].astype(str) >= start_date) & (dff['Connected On'].astype(str) <= end_date)]
    month_counts = dff['month'].value_counts()
    
    # Re-order months for correct plotting
    month_order = list(calendar.month_abbr)[1:]
    plot_df = pd.DataFrame({'month': month_order}).merge(
        month_counts.rename('Total following').reset_index(),
        on='month',
        how='left'
    ).fillna(0)
    plot_df['month'] = pd.Categorical(plot_df['month'], categories=month_order, ordered=True)
    plot_df = plot_df.sort_values('month')

    fig_line = px.line(
        plot_df, x='month', y='Total following', template='ggplot2', title="Following Activity by Month",
        labels={'month': 'Month', 'Total following': 'Users Followed'}
    )
    fig_line.update_traces(mode="lines+markers", fill='tozeroy', line={'color':'#E1306C'}) # IG pink/red
    fig_line.update_layout(margin=dict(l=20, r=20, t=30, b=20), xaxis_title=None)
    return fig_line

# Callback for Instagram Bar Chart (Top Followed Users - using 'value' column)
@app.callback(
    Output('ig-bar-chart','figure'),
    [Input('ig-date-picker-start','date'),
     Input('ig-date-picker-end','date')],
)
def update_ig_bar(start_date, end_date):
    if not (start_date and end_date):
        return {}

    dff = df_ig_cnt.copy()
    dff = dff[(dff['Connected On'].astype(str) >= start_date) & (dff['Connected On'].astype(str) <= end_date)]
    dff = dff.dropna(subset=['value']) # 'value' holds the user names
    
    # Use 'value' as the field to count for top connections
    user_counts = dff['value'].value_counts().head(6).reset_index()
    user_counts.columns = ['User', 'Follows']

    fig_bar = px.bar(
        user_counts, x='Follows', y='User', template='ggplot2', orientation='h', 
        title="Top 6 Followed Users/Pages (Needs better data)",
        labels={'Follows': 'Count', 'User': 'User/Page'}
    )
    fig_bar.update_yaxes(tickangle=0)
    fig_bar.update_layout(margin=dict(l=20, r=20, t=30, b=20), yaxis_title=None)
    fig_bar.update_traces(marker_color='#C13584') # IG purple

    return fig_bar

# Callback for Instagram Pie Chart (DMs Sent & Received)
@app.callback(
    Output('ig-pie-chart','figure'),
    [Input('ig-date-picker-start','date'),
     Input('ig-date-picker-end','date')],
)
def update_ig_pie(start_date, end_date):
    if not (start_date and end_date):
        return {}

    dff = df_ig_msg.copy()
    dff = dff[(dff['DATE'].astype(str) >= start_date) & (dff['DATE'].astype(str) <= end_date)]
    
    # Assuming 'Sean Calvin Soliven' is the user, as specified in your original code, and using messages__sender_name column
    user_name = 'Sean Calvin Soliven' # Replace with actual user name if known
    
    # Note: Using 'messages__sender_name' based on your IG code logic, assuming it identifies the sender
    msg_sent = len(dff[dff['messages__sender_name'] == user_name])
    msg_rcvd = len(dff[dff['messages__sender_name'] != user_name])
    
    if msg_sent + msg_rcvd == 0:
        return {}

    fig_pie = px.pie(names=['Sent','Received'], values=[msg_sent, msg_rcvd],
                     template='ggplot2', title="Direct Messages Sent & Received"
    )
    fig_pie.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_pie.update_traces(marker_colors=['#F56040','#833AB4']) # IG colors

    return fig_pie

# Callback for Instagram Word Cloud (Positions - Not applicable for IG, using a placeholder)
@app.callback(
    Output('ig-wordcloud','figure'),
    [Input('ig-date-picker-start','date'),
     Input('ig-date-picker-end','date')],
)
def update_ig_wordcloud(start_date, end_date):
    # Instagram connection data does not typically have a 'Position' field like LinkedIn.
    # As a placeholder, we can try to use a word cloud of user names or return a default plot.
    
    if not (start_date and end_date):
        return {}

    dff = df_ig_cnt.copy()
    dff = dff[(dff['Connected On'].astype(str) >= start_date) & (dff['Connected On'].astype(str) <= end_date)]
    dff_usernames = dff['value'].dropna().astype(str)

    if dff_usernames.empty:
         # Return a blank image placeholder if no data is found
        return px.imshow(np.zeros((10,10)), color_continuous_scale='gray', title="No Username Data Found").update_xaxes(visible=False).update_yaxes(visible=False).update_layout(margin=dict(l=20, r=20, t=40, b=20))

    # Generate word cloud from usernames for the period
    my_wordcloud = WordCloud(background_color='white', width=800, height=400, colormap='magma').generate(' '.join(dff_usernames))
    img_array = my_wordcloud.to_array()

    fig_wordcloud = px.imshow(img_array, title="Most Frequently Followed Users")
    fig_wordcloud.update_layout(margin=dict(l=20, r=20, t=40, b=20), xaxis_visible=False, yaxis_visible=False)

    return fig_wordcloud


# --- 7. RUN THE APP ---

app.run(mode='jupyterlab', port=8009)
webbrowser.open("http://127.0.0.1:8009")

C:\Users\soliv\AppData\Local\Temp\ipykernel_5008\33532961.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_li_cnt["Connected On"] = pd.to_datetime(df_li_cnt["Connected On"], errors='coerce')
C:\Users\soliv\AppData\Local\Temp\ipykernel_5008\33532961.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_li_invite["Sent At"] = pd.to_datetime(df_li_invite["Sent At"], errors='coerce')


True